In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, GridSearchCV

Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм".

In [3]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [4]:
X = newsgroups.data
y = newsgroups.target

Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [5]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [6]:
grid = {"C": np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel="linear", random_state=241)
gs = GridSearchCV(clf, grid, scoring="accuracy", cv=cv, verbose=1, n_jobs=-1)
gs.fit(X, y)

C = gs.best_params_
print(C["C"]) 

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:   44.2s finished


1.0


Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [7]:
clf = SVC(C=C["C"], kernel="linear", random_state=241)
clf.fit(X, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=241, shrinking=True, tol=0.001,
    verbose=False)

Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [29]:
words = np.array(vectorizer.get_feature_names())
weights = pd.Series(data=clf.coef_.data, index=words[clf.coef_.indices], name="weight")
weights.index.name = "word"

max_weights = weights.abs().sort_values(ascending=False).head(10)
max_weights = sorted(max_weights.index)

In [30]:
f = open('1.txt', 'w')
f.write(','.join(map(str, max_weights)))
f.close()